In [1]:
import os
import keras
from keras.models import model_from_json
from scipy import misc, ndimage
import imageio
import skimage
import numpy as np
from keras.layers import Conv2D, Activation, Reshape
from keras.models import Model
from keras.models import Sequential
from layers_builder import Interp
import layers_builder as layers
from keras.optimizers import SGD
from python_utils.preprocessing import data_generator_s31

from python_utils.callbacks import callbacks
keras.__version__

Using TensorFlow backend.


'2.2.0'

### Load model

In [ ]:
# #load pretrain pspnet101_cityscape

# print("Loanding model...")
# weights = "pspnet101_cityscapes"

# json_path = os.path.join("weights", "keras", weights + ".json")
# h5_path = os.path.join("weights", "keras", weights + ".h5")

# with open(json_path, 'r') as file_handle:
#     model = model_from_json(file_handle.read())
# model.load_weights(h5_path)
# x = model.get_layer("dropout_1").output
# x = Conv2D(5, (1, 1), strides=(1, 1), name="conv6")(x)
# x = Interp([233, 233])(x)
# x = Activation('softmax', name="activation_110")(x)
# new_model_1 = Model(inputs=model.input, outputs=x)

# new_model = layers.build_pspnet(nb_classes=5,
#                             resnet_layers=101,
#                             input_shape=(233,233))

# for layer in new_model.layers[:408]:
#     layer.trainable = False
# print(new_model.summary())


In [2]:
# load own pretrain

print("Loanding model...")
weights = "pspnet101_cityscapes"

h5_path = "/home/thaovt7/workspace/PSPNet-Keras-tensorflow/weights_train/weight_cityscapes_last_layer/weights.150-20.10.h5"
model = layers.build_pspnet(nb_classes=5,
                            resnet_layers=101,
                            input_shape=(713,713))
model.load_weights(h5_path)

new_model = layers.build_pspnet(nb_classes=5,
                            resnet_layers=101,
                            input_shape=(233,233))
new_model.set_weights(model.get_weights())
for layer in new_model.layers[:408]:
    layer.trainable = False
print(new_model.summary())


Loanding model...
Building a PSPNet based on ResNet 101 expecting inputs of shape (713, 713) predicting 5 classes
PSP module will interpolate to a final feature map size of (90, 90)
Building a PSPNet based on ResNet 101 expecting inputs of shape (233, 233) predicting 5 classes
PSP module will interpolate to a final feature map size of (30, 30)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 233, 233, 3)  0                                            
__________________________________________________________________________________________________
conv1_1_3x3_s2 (Conv2D)         (None, 117, 117, 64) 1728        input_2[0][0]                    
__________________________________________________________________________________________________
conv1_1_3x3_s2_bn (BatchNormali (None, 117, 117, 64) 256    

/home/thaovt7/anaconda3/envs/pspnet/lib/python3.5/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


# Train

In [ ]:
# import tensorflow as tf

In [ ]:
# def build_PSPNet_loss(class_weights=None):
#     """    Return a PSPNet loss function corresponding to class_weights    """
#     def PSPNet_loss(y_true, y_pred):
#         y_true = tf.cast(y_true, tf.float32)
#         y_pred = tf.cast(y_pred, tf.float32)
#         weighted_values =  class_weights * y_true * tf.log(y_pred)
#         pixelwise_cross_entropy = tf.reduce_sum(weighted_values,axis=2)
#         loss = - tf.reduce_sum(pixelwise_cross_entropy)
#         return loss
#     return PSPNet_loss

In [ ]:
from keras import backend as K
def build_PSPNet_loss(class_weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    class_weights = K.variable(class_weights)
        
    def PSPNet_loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
#         y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * class_weights
#         loss = -K.sum(loss, -1)
        loss = -K.sum(loss)/713/713
        return loss
    
    return PSPNet_loss

In [ ]:
# class_weights_value = np.zeros((16,121,121,5))
# weights = np.array([8.6,63.1,268.,63.1,1.])
# class_weights_value[:,:,None] = weights

class_weights_value = np.zeros((20,233,233,5))
weights = np.array([7.9,58.5,284.,58.5,1.])
class_weights_value[:,:,None] = weights

# class_weights_value = np.array([7.9,58.5,284.,58.5,1.])

In [ ]:
# pspnet_loss = build_PSPNet_loss(class_weights_value.astype(np.float32))
pspnet_loss = build_PSPNet_loss(class_weights_value)

In [ ]:
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
new_model.compile(optimizer=sgd, loss=pspnet_loss, metrics=['accuracy'])

In [ ]:
datadir = '/home/thaovt7/workspace/Dataset'
batchsize = 10
input_size = (233,233)
nb_classes = 5
dataset_len = len(os.listdir(os.path.join(datadir, 'imgs')))
print("Create data generator...")
train_generator, val_generator = data_generator_s31(
    datadir=datadir, batch_size=batchsize, input_size=input_size, nb_classes=nb_classes)

In [ ]:
logdir = './logs'
initial_epoch = 0
new_model.fit_generator(
         generator=train_generator,
         epochs=150, verbose=True, steps_per_epoch=10,
         callbacks=callbacks(logdir),
        initial_epoch=initial_epoch)

### Test prediction

In [ ]:
from skimage.transform import rescale, resize

img_path = "/home/thaovt7/workspace/Dataset/imgs_test/cityscapes/aachen_000003_000019_leftImg8bit.png"
input_size = 121
print("Processing image {}".format(img_path))
img = misc.imread(img_path, mode='RGB')
# img = imageio.imread(img_path, pilmode="RGB")
cimg = misc.imresize(img, (input_size, input_size))
# cimg = resize(img, (input_size, input_size), mode='reflect', anti_aliasing=False)

In [ ]:
img.shape

In [ ]:
# These are the means for the ImageNet pretrained ResNet
DATA_MEAN = np.array([[[123.68, 116.779, 103.939]]])  # RGB order

cimg = cimg - DATA_MEAN
cimg = cimg[:, :, ::-1]  # RGB => BGR
cimg = cimg.astype('float32')

In [ ]:
print("Predicting...")
prediction = model.predict(np.expand_dims(cimg, 0))[0]

In [ ]:
h_ori, w_ori = img.shape[:2]
h, w = prediction.shape[:2]
prediction = ndimage.zoom(prediction, (1. * h_ori / h, 1. * w_ori / w, 1.),
                                 order=1, prefilter=False)

In [ ]:
cm = np.argmax(prediction, axis=2)

In [ ]:
np.unique(cm)

In [ ]:
from python_utils import utils
color_cm = utils.add_color(cm)


# Display images

### display mask

In [ ]:
from PIL import Image
mask_im = Image.fromarray(np.uint8(color_cm))
mask_im.show()